In [1]:
import pandas as pd

df = pd.read_csv('subset_tweets_preprocessed.csv')
df

,tweet_id,text,account,domain,background,label
0,1603396689552965635,fútbolplaya copaconosur 🇦🇷🇧🇴🇺🇾🇨🇱 argentina enf...,@Argentina,PERSON,SPORT,ARG
1,1608210531713695745,opinión en cuestiones de igualdad de género sa...,@publico_es,ORG,NEW,ES
2,1608881936512032768,picaenpunta el club inglés quiere llevarse al ...,@cronica,ORG,NEW,ARG
3,1550218833729933312,mira este video sus segundos importan y merece...,@AlejandroSanz,PERSON,ART,ES
4,1608743487448522752,la transparencia de ratajkowski e increíble ha...,@laSextaTV,ORG,NEW,ES
...,...,...,...,...,...,...
9979,1584567972622127106,basta,@sanchezcastejon,PERSON,POL,ES
9980,1608146947218591744,china reabre sus fronteras después de tres año...,@publico_es,ORG,NEW,ES
9981,1603662631985770497,foto de un momento de riña entre señorías y es...,@gabrielrufian,PERSON,POL,ES
9982,1585738322386644994,hoy con maxigallucci estuvimos en avellaneda e...,@horaciorlarreta,PERSON,POL,ARG


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    DistilBertConfig,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments,
)
from datasets import Dataset
from datasets.load import load_dataset, load_metric


# Opening Dataset
df = pd.read_csv('subset_tweets_preprocessed.csv')

# Load the dataset from the Hugging Face datasets library
dataset = load_dataset('csv', data_files='subset_tweets_preprocessed.csv', delimiter=',')

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("dccuchile/distilbert-base-spanish-uncased")

# Encoding Data
def process_data(row):
    text = row['text']
    text = str(text)
    text = ' '.join(text.split())
    encodings = tokenizer(text, padding="max_length", truncation=True, max_length=128)
    label = row['label']
    if label == 'ES':
        label = 1
    else:
        label = 0
    encodings['label'] = label
    encodings['text'] = text
    return encodings

processed_data = [process_data(row) for _, row in df.iterrows()]

# Shuffle the dataset
import random
random.shuffle(processed_data)

# Splitting Dataset
train, test = train_test_split(processed_data, test_size=0.2)

train_dict = {k: [example[k] for example in train] for k in train[0]}
test_dict = {k: [example[k] for example in test] for k in test[0].keys()}


train_dataset = Dataset.from_dict(train_dict)
test_dataset = Dataset.from_dict(test_dict)

# Config and Model
config = DistilBertConfig.from_pretrained(
    "dccuchile/distilbert-base-spanish-uncased", num_labels=2, output_hidden_states=True
)
model = DistilBertForSequenceClassification.from_pretrained(
    "dccuchile/distilbert-base-spanish-uncased", config=config,
)

# Data Collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

# Metric and Metric computation function
metric = load_metric("f1")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions
    if isinstance(preds, tuple):
        preds = preds[0]
    preds = np.argmax(preds, axis=-1)
    f1 = metric.compute(predictions=preds, references=labels, average="weighted")
    return f1


# Training Arguments
training_args = TrainingArguments(
    output_dir="./new_stuff",
    evaluation_strategy="steps",
    eval_steps=10,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

# Evaluation
eval_results = trainer.predict(test_dataset)
eval_metric = compute_metrics(eval_results)
print(eval_metric)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/LENOVO/.cache/huggingface/datasets/csv/default-0157baece0c0bcda/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Some weights of the model checkpoint at dccuchile/distilbert-base-spanish-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at dccuchile/distilbert-base-spanish-uncased and are newly initialized: ['classifier.bias', 

Step,Training Loss,Validation Loss,F1
10,0.691200,0.693227,0.335769
20,0.671400,0.681500,0.399781
30,0.707900,0.671393,0.472199
40,0.655500,0.664864,0.490269
50,0.695600,0.645997,0.630942
60,0.650500,0.622959,0.687322
70,0.625500,0.595674,0.726662
80,0.571700,0.567602,0.723555
90,0.565600,0.543488,0.727104
100,0.582000,0.527544,0.738520


{'f1': 0.8272395614092006}
